In [1]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 167 kB of archives.
After this operation, 558 kB of additional disk space will be used.
Get:1 http://mirror.yandex.ru/ubuntu bionic/main amd64 unzip amd64 6.0-21ubuntu1 [167 kB]
Fetched 167 kB in 0s (1049 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 6778 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1_amd64.deb ...
Unpacking unzip (6.0-21ubuntu1) ...
Setting up unzip (6.0-21ubuntu1) ...


In [5]:
!unzip quora.csv.zip

Archive:  quora.csv.zip
  inflating: quora.csv               
  inflating: __MACOSX/._quora.csv    


In [8]:
!pip install transformers sklearn pandas

     |████████████████████████████████| 10.0 MB 670 kB/s eta 0:00:01


In [3]:
# стандартные библиотеки
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd


# pytortch и huggingface 
import torch
from transformers.modeling_auto import AutoModel
from transformers import AutoTokenizer


Возьмем данные с соревнования по определению токсичных вопросов на Quora.

In [4]:
data = pd.read_csv('quora.csv')

In [5]:
data.drop('qid', axis=1, inplace=True)

In [6]:
data.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


Разобьем данные на трейн и тест.

In [7]:
train, test = train_test_split(data, test_size=0.05, stratify=data.target)

In [8]:
train.shape

(1240815, 2)

In [9]:
test.shape

(65307, 2)

### Загружаем предобученную модель из huggingface transformers

Список всех доступных моделей можно найти тут - https://huggingface.co/models  
А вот тут основные с описанием - https://huggingface.co/transformers/pretrained_models.html

Данные у нас на английском, поэтому мы возьмем оригинального Берта (остальные модели загружаются также)

In [10]:
# # Eng Bert
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model_bert = AutoModel.from_pretrained('bert-base-cased')

In [11]:
# # Rubert - от IPavlov
# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# model_bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to(torch.device('cuda'))

In [22]:
# # Multilingual Bert - от гугла
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model_bert = AutoModel.from_pretrained('bert-base-multilingual-cased')

Мы загружаем не только модель, а еще и токенайзер, т.е. свою предобработку нам писать не нужно

In [12]:
train.loc[0, 'question_text']

'How did Quebec nationalists see their province as a nation in the 1960s?'

Перевести токены в индексы очень просто

In [13]:
tokenizer.encode('How did Quebec nationalists see their province as a nation in the 1960s?')

[101,
 1731,
 1225,
 5181,
 25170,
 1267,
 1147,
 3199,
 1112,
 170,
 3790,
 1107,
 1103,
 3266,
 136,
 102]

В этих моделях как правило используется BPE, но на стандартных словах этого не видно и может показаться, что все просто разбивается по пробелам

In [14]:
# переводим индекс токена обратно в текст
encoded = tokenizer.encode('How did Quebec nationalists see their province as a nation in the 1960s?')
[tokenizer.decode([x]) for x in encoded]

['[CLS]',
 'How',
 'did',
 'Quebec',
 'nationalists',
 'see',
 'their',
 'province',
 'as',
 'a',
 'nation',
 'in',
 'the',
 '1960s',
 '?',
 '[SEP]']

Сделаем опечатки словах, чтобы увидеть, что это все-таки BPE

In [15]:
# переводим индекс токена обратно в текст
encoded = tokenizer.encode('Hw did Qubec nazionalists see their province asa nation in the 1960s?')
[tokenizer.decode([x]) for x in encoded]

['[CLS]',
 'H',
 '##w',
 'did',
 'Q',
 '##ube',
 '##c',
 'na',
 '##zio',
 '##nal',
 '##ists',
 'see',
 'their',
 'province',
 'as',
 '##a',
 'nation',
 'in',
 'the',
 '1960s',
 '?',
 '[SEP]']

Индексы можно напрямую передавать в модель.

In [16]:
text = 'How did Quebec nationalists see their province as a nation in the 1960s?'

text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])

output = model_bert(text_ids)

На выходе мы получим tuple из двух элементов. 

Первый элемент - состояния енкодера для каждого из элементов последовательности

In [17]:
output[0].size() # в пайторче вместо .shape используется size()

torch.Size([1, 16, 768])

Второй - состояние енкодера на первом элементе, пропущенное через активацию (обычно этот элемент не используют)

In [18]:
output[1].size()

torch.Size([1, 768])

Обычно в задачах используют либо состояние первого элемента

In [19]:
output[0][:,0].size()

torch.Size([1, 768])

Либо усредненное состояние 

In [20]:
torch.mean(output[0], axis=1).size()

torch.Size([1, 768])

Полученные эмбеддинги уже можно использовать для какой-нибудь кластеризации или поиска похожих. А если есть разметка, то можно обучить на этих векторах стандартную модель из sklearn или даже дообучить всего Берта под конкретную задачу!

Давайте попробуем дообучить (fine-tune) модель на данных Quora

Нужно прописать генератор бачтей и саму модель.

In [21]:
def generate_batches_train(train, batch_size=32):
    # очень простая функция
    # перемешиваем датасет и выдает тексты и метки классов по кусочкам (в 32 например)
    batch_q = []
    batch_a = []
    
    while True: # чтобы сам никогда не заканчивался
        
        for i, (q,a) in enumerate(train.sample(frac=1.).values):
            
            batch_q.append(q)
            batch_a.append(a)

            if len(batch_q) == batch_size:

                yield (batch_q,
                       np.array(batch_a, dtype=np.int32))
                batch_q = []
                batch_a = []

                
def generate_batches_test(test, batch_size=32):
    # тоже самое только без бесконечной генерации
    batch_q = []
    batch_a = []
        
    for i, (q,a) in enumerate(test.values):

        batch_q.append(q)
        batch_a.append(a)

        if len(batch_q) == batch_size:

            yield (batch_q,
                   np.array(batch_a, dtype=np.int32))
            batch_q = []
            batch_a = []

In [22]:
gen = generate_batches_train(train, 8)

In [23]:
next(gen)

(['Is it possible to get a poultry job in Canada from Nigeria?',
  'What did the Brazilian Defenders do and not get red cards to James Rodriguez that made him weep?',
  'Are the apolipoproteins in a lipoprotein hydrophobic or amphipathic?',
  'Why is London a Heaven for loan defaulters of India?',
  'What policies Mexico has adopted to resettle and reintegrate displaced population through improving food security?',
  'Why do people smile while taking photograghs?',
  'How do I convert 14 v AC supply into DC?',
  'Is Rugby union dangerous, and if so, can it be made safer?'],
 array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))

Теперь модель. До этого мы не использовали pytorch, но не пугайтесь. У нас не такая сложная модель и различия все-таки не такие большие

In [32]:
import torch.nn as nn

class CLF(nn.Module):
    
    def __init__(self, pretrained_model):
        super().__init__()          
        # этот шаг похож на определение модели в керасе через Functional API
        self.tokenizer = tokenizer # токенизатор
        self.pretrained_model = pretrained_model # предобученная модель

        self.fc = nn.Linear(768, 1) # аналог Dense слоя без активации (но нужно указать shape - 768 на вход 1 на выход)
        self.act = nn.Sigmoid() # активация
        
    # тут задает как слои будут применятся к входным данным
    def forward(self, texts):
        # токенизируем
        texts_ids = [torch.tensor(self.tokenizer.encode(t, add_special_tokens=True)) for t in texts]
        # делаем паддинг
        texts_ids = torch.nn.utils.rnn.pad_sequence(texts_ids, batch_first=True, 
                                                    padding_value=self.tokenizer.pad_token_id).to(torch.device('cuda'))
        # чтобы нули не учитывались считаем маску
        mask = (texts_ids != tokenizer.pad_token_id).long()
        
        # прогоняем через BERT
        hidden = self.pretrained_model(texts_ids, attention_mask=mask)[0]

        # берем самое первое состояние и применяем к нему линейный слой и активацию
        dense_outputs=self.fc(hidden[:,0] )
        outputs=self.act(dense_outputs)
        
        return outputs

Задаем оптимизатор, лосс и метрику

In [33]:
model = CLF(model_bert)

In [34]:
import torch.optim as optim

#
optimizer = optim.Adam(model.parameters(), lr=1e-5) # тот же адам
criterion = nn.BCELoss() # binary_crossentropy

#обычная accuracy
def accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    


Модель готова

В торче больше вещей нужно делать вручную. Например, нужно посылать модель или тензоры на видеокарту. Поэтому к модели добавится .to(device)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [36]:
model = model.to(device)
criterion = criterion.to(device)

Обучающий цикл тоже надо прописывать подробнее

In [37]:
iterator = generate_batches_train(train, 20)

In [38]:
epoch_loss = []
epoch_acc = []

# включаем training mode
model.train()  

for i, batch in enumerate(iterator):

    #обнуляем градиенты - не так важно сразу понимать
    optimizer.zero_grad()   


    texts, labels = batch[0], batch[1]   

    #пропускаем тексты через модель (вызываем forward) 
    predictions = model(texts).squeeze()  

    # считаем лосс
    loss = criterion(predictions, torch.Tensor(labels).to(device))        

    # считаем accuract
    acc = accuracy(predictions, torch.Tensor(labels).to(device))   

    # делаем backprop
    loss.backward()       

    # обновляем веса
    optimizer.step()      

    #loss and accuracy
    epoch_loss.append(loss.item())  
    epoch_acc.append(acc.item())
    
    
    if not ((i)+1)%100:
        print(np.mean(epoch_loss), np.mean(epoch_acc))


0.24015134515240788 0.9100000078231096
0.19485027224291115 0.9270000076666475
0.17506930229254067 0.935166672890385
0.16318675937596708 0.9391250059194863
0.153706934761256 0.9407000060230494
0.14777731076154546 0.9422500057145953
0.1425283735388491 0.9440000051366432
0.14049461299582616 0.9445625052694231
0.13800016964759884 0.946277782726619
0.1362622226185631 0.9468000043705106
0.13417358377884905 0.9475000043958426
0.1334997557090052 0.9481250042282044
0.132444141268766 0.9485769270990904
0.13194980433110945 0.9483214326468962
0.13167523896383743 0.9483666707724333
0.13096283352177124 0.9484062540577725
0.13120313635356176 0.948264710066073
0.12941021727307492 0.9489444486010405
0.12803378262814427 0.9495789514129099
0.12734245934465435 0.9496500040329993
0.1265202674105586 0.9497380992663759
0.12652335708939724 0.9498181858963587
0.12582117192467432 0.9501739170531863
0.12521345609071433 0.9503125039767474
0.12455962450830266 0.9504800039798021
0.12385731470636809 0.95082692694492

0.10298817169930165 0.958652584250894
0.10297050414939855 0.9586495348034757
0.10293950775664214 0.9586511648813653
0.10284701997789347 0.9587106502280329
0.10277028941899143 0.9587396334067055
0.10270885672019218 0.9587614699518052
0.10263934344788439 0.958778540876154
0.10262966550615171 0.9587704566070302
0.10258122054822595 0.9588009070301622
0.10259138114684836 0.9588018038496375
0.10257205227574212 0.9588183876942226
0.1025493865831491 0.9588258949005312
0.10252639574464928 0.9588644464734528
0.10246536479954324 0.9588893825583885
0.10237188566902279 0.9589273147964399
0.10231718950886014 0.9589407914880206
0.10231233312974594 0.9589344998195312
0.1023487641258803 0.9589282628689771


KeyboardInterrupt: 

In [43]:
# иногда будут возникать ошибки с кудой, можно попробовать запустить такой код
# или попробовать батч сайз поменьше
import gc
gc.collect()
torch.cuda.empty_cache()

Тестируем модель

In [44]:
test_loss = []
test_acc = []

# выключаем training mode
model.eval()  
iterator_test = generate_batches_test(test, 20)
for i, batch in enumerate(iterator_test):

    texts, labels = batch[0], batch[1]   

    # пропускаем тексты через модель (вызываем forward) 
    predictions = model(texts).squeeze()  

    # считаем лосс
    loss = criterion(predictions, torch.Tensor(labels).to(device))        

    # считаем accuract
    acc = accuracy(predictions, torch.Tensor(labels).to(device))   

    # ничего не обновляем   
    
    test_loss.append(loss.item())  
    test_acc.append(acc.item())


In [46]:
print(np.mean(test_loss), np.mean(test_acc))

0.09276314006000899 0.9620826965451789


In [75]:
torch.save(model, 'en_bert_finetuned_quora')

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CLF. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
